# Template for segmented MNL models

Sam Maurer, Sep 2018

This notebook tests and demonstrates the segmented MNL models added in [UrbanSim Templates PR #45](https://github.com/UDST/urbansim_templates/pull/45).

In [1]:
import numpy as np
import pandas as pd

import orca
import urbansim_templates

from urbansim_templates import modelmanager
from urbansim_templates.models import SegmentedLargeMultinomialLogitStep

urbansim_templates.__version__

'0.1.dev13'

### Set up testing data

In [2]:
d1 = {'oid': np.arange(10), 
      'group': np.random.choice(['A','B','C'], size=10),
      'obsval': np.random.random(10),
      'choice': np.random.choice(np.arange(20), size=10)}

d2 = {'aid': np.arange(20), 
      'altval': np.random.random(20)}

obs = pd.DataFrame(d1).set_index('oid')
orca.add_table('obs', obs)

alts = pd.DataFrame(d2).set_index('aid')
orca.add_table('alts', alts)

In [3]:
obs

,group,obsval,choice
oid,,,
0,A,0.829421,16
1,C,0.370273,10
2,A,0.709472,6
3,C,0.612958,0
4,B,0.274402,6
5,B,0.370351,9
6,C,0.730241,13
7,A,0.699012,9
8,C,0.627819,10


### Test model features

In [4]:
m = SegmentedLargeMultinomialLogitStep()

In [5]:
m.defaults.choosers = 'obs'
m.defaults.alternatives = 'alts'
m.defaults.choice_column = 'choice'
m.defaults.model_expression = 'obsval + altval'
m.segmentation_column = 'group'

In [6]:
m.build_submodels()

Building submodels for 3 categories: ['A' 'B' 'C']


In [13]:
for mod in m.submodels:
    print(mod.chooser_filters)

group == 'A'
group == 'B'
group == 'C'


In [12]:
m.fit_all()

                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:              3
Model:         Multinomial Logit   Df Residuals:                  0
Method:       Maximum Likelihood   Df Model:                      3
Date:                 2018-09-21   Pseudo R-squ.:             0.011
Time:                      14:36   Pseudo R-bar-squ.:        -0.328
AIC:                      23.467   Log-Likelihood:           -8.734
BIC:                      20.763   LL-Null:                  -8.833
               coef   std err         z     P>|z|   Conf. Int.
--------------------------------------------------------------
Intercept   -0.0000    11.609    -0.000     1.000             
obsval       0.0000    17.768     0.000     1.000             
altval       0.7955     3.384     0.235     0.814             
                  CHOICEMODELS ESTIMATION RESULTS                  
Dep. Var.:                chosen   No. Observations:              2
Model

/Users/maurer/Dropbox/Git-imac/udst/choicemodels/choicemodels/mnl.py:235: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  chosen = np.reshape(self._df[[self._choice_col]].as_matrix(),
/Users/maurer/Dropbox/Git-imac/udst/choicemodels/choicemodels/mnl.py:238: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  log_lik, fit = mnl_estimate(model_design.as_matrix(), chosen, self._numalts)
